In [ ]:
import numpy as np
import torch
from grid2vec import *
import json
import pandas as pd
from utils import Timer, copy_file
from joblib import Parallel, delayed
from traj2grid import Traj2Grid
import traj_dist.distance as tdist
from logging import raiseExceptions
from parameters import *
import numpy as np
import time

timer = Timer()

In [ ]:
str_grid2idx = json.load(open("data/str_grid2idx_300.json"))
grid2idx = {eval(c): str_grid2idx[c] for c in list(str_grid2idx)}
print(len(grid2idx))

In [ ]:
max_lon = 0
max_lat = 0
min_lon = 999
min_lat = 999
timer.tik()
for i in range(1, 31):
    df = pd.read_csv("data/full/gps_201611"+str(i).zfill(2),
                     names=["name", "order_id", "time", "lon", "lat"],
                     usecols=["lon", "lat"])
    max_lon = df['lon'].max() if df['lon'].max() > max_lon else max_lon
    min_lon = df['lon'].min() if df['lon'].min() < min_lon else min_lon
    max_lat = df['lat'].max() if df['lat'].max() > max_lat else max_lat
    min_lat = df['lat'].min() if df['lat'].min() < min_lat else min_lat
    timer.tok(str(i).zfill(2))
    print(max_lon, min_lon, max_lat, min_lat)

In [3]:
from grid2vec import *

with open('data/str_grid2idx_400_44612.json') as f:
    str_grid2idx = json.load(f)
    f.close()
grid2idx = {eval(c): str_grid2idx[c] for c in list(str_grid2idx)}

dataset = GridEmbeddingDataset(grid2idx, window_size=20, neg_rate=100)
model = Grid2Vec(len(dataset), 256)
cp = torch.load('model/checkpoint_256_298_0_0.179.pth')
model.load_state_dict(cp['model'])
weight2 = np.load("model/grid_embedding_256_0.83.npy")
weight3 = np.load("model/grid_embedding_256_0.79.npy")
np.save("model/grid_embedding_256_",model.input_embedding())
# print(evaluate_grid2vec(model.input_embedding(), dataset, test_num=500))
# print(evaluate_grid2vec(weight2, dataset, test_num=500))
# print(evaluate_grid2vec(weight3, dataset, test_num=500))


0.8272
0.8229
0.7802


In [88]:
import torch

lens = torch.randperm(4)+1
lens = torch.tensor([3,4,5])

In [10]:
bo = torch.rand(3,6)
bo < 0.5

tensor([[ True, False,  True, False,  True, False],
        [ True, False,  True,  True,  True, False],
        [ True,  True, False, False,  True, False]])

In [11]:
b = torch.rand(3,6,2) # bsz, seq_len, emb_dim
b[bo<0.5] = 0
b

tensor([[[0.0000, 0.0000],
         [0.9497, 0.2976],
         [0.0000, 0.0000],
         [0.1735, 0.1189],
         [0.0000, 0.0000],
         [0.2856, 0.5854]],

        [[0.0000, 0.0000],
         [0.3486, 0.5454],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0569, 0.7157]],

        [[0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.7021, 0.6735],
         [0.8773, 0.5123],
         [0.0000, 0.0000],
         [0.0764, 0.3002]]])